In [1]:
# Import dependencies
import numpy as np
import pandas as pd
import tweepy
import time
import json

import boto3
import botocore

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.externals import joblib

from keras.models import load_model

Using TensorFlow backend.


In [2]:
# import the model from AWS
BUCKET_NAME = 'mlsomm' # replace with your bucket name
KEY = 'ML_Somm.h5' # replace with your object key

s3 = boto3.resource('s3')

try:
    s3.Bucket(BUCKET_NAME).download_file(KEY, 'somm_model.h5')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [5]:
# Twitter API Keys
consumer_key = "AoUHxKdOO6aein5z81cdCDZxs"
consumer_secret = "MSEMsxbmglFzBUQsQpNoaS98GCvwDG1xDRrB9hrrvJjD1mrwr4"
access_token = "922955241078841344-oPc8kbdukcySJbL13OroEOYlF8CB9ZE"
access_token_secret = "nAc5b4zmxJ7hf1vC0HjNTZdISWglfrzqbCVnfrwuIdmHU"

# Twitter Credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [9]:
responses = ['Hey there! You might want to try a ', 'That sounds like a ', "I think you'd love a "]

def get_mentions(last_id, wine_description):
    mentions = api.mentions_timeline(since_id = last_id, count=1, result_type="recent")
    last_id = mentions[0]['id']
    for mention in mentions:
        wine_description = mention['text']
        target_user = '@' + mention['user']['screen_name']
        print("Wine description: " +  wine_description)
        print("Requested by: " + target_user)
#         count = 0 
#         while count < len(responses):
#             for i in responses:
#                 api.update_status(i + "" + target_user)
#                 count+= 1 
    return last_id, wine_description, target_user

In [13]:
last_id = None
wine_description = None
count = 0
while count<5:
    try:
        last_id, wine_description, target_user = get_mentions(last_id, wine_description)
        wine = runModel(wine_description)
        out = get_wine_variety_response(wine)
        tweetBack(out, target_user)
        print(out)
        count +=1
    except:
        time.sleep(1)
    #print(since_id_)

Wine description: @mberkow2017 Ripe cherries and strawberries, spice, and vanilla aromas. This bold wine is well balanced, with earth… https://t.co/EiLQSJ9JbR
Requested by: @tiwaamisu
['Barbera']
['The highest rated Barberas are from these wineries: Braida di Giacomo Bologna, Franco M. Martinetti, or Cantina Iuli. Avg Price: $25.22', 'The least expensive Barberas are from these wineries: Cascina Guido Berta, F.lli Trinchero, or Vietti. Avg Price: $24.15']


KeyboardInterrupt: 

In [2]:
def runModel(wine_description):
    vectorizer = joblib.load('vectorizer.pkl')
    label_encoder = joblib.load('label_encoder.pkl') 



    # load the model
    model = load_model("somm_model.h5")
    # model = load_model("ML_Somm.h5")

    # test the data
    test = [wine_description]
    test_vec = vectorizer.transform(test)
    encoded_test = model.predict_classes(test_vec)
    predict_label = label_encoder.inverse_transform(encoded_test)
    print(predict_label)
    
    wine_type = predict_label[0]
    
    return wine_type

In [3]:
def get_wine_variety_response (variety):
    #read the CSV file
    sommData = pd.read_csv('Data/no_dups_wine_data.csv')
    
    output_tweets = []
    
    #find all the wineries for a particular variety to add to the output tweet
    variety_prediction = variety
    var_list = sommData.loc[sommData['variety'] == variety_prediction]
    
    #get the highest rated wines
    sorted_df = var_list.sort_values(["points"], ascending=[False])
    #sorted_df['winery'].head()
    wineries = sorted_df[['winery','price']]
    wineries['winery'].unique()[0:3]
    
    #get the average price for all wines in the TOP 3 wineries
    selected_wineries = wineries[wineries['winery'].isin(wineries['winery'].unique()[0:3])]
    avg_price = selected_wineries['price'].mean()
    
    wines = ("The highest rated " + variety_prediction + "s are from these wineries: " + wineries['winery'].unique()[0] + ", " \
        + wineries['winery'].unique()[1] + ", or " + wineries['winery'].unique()[2] +". Avg Price: $%.2f" % avg_price)
    output_tweets.append(wines)
    
    #least expensive
    sorted_df = var_list.sort_values(["price"], ascending=[True])
    #sorted_df['winery'].head()
    wineries = sorted_df[['winery','price']]
    
    #get the average price for all wines in the BOTTOM 3 wineries
    selected_wineries = wineries[wineries['winery'].isin(wineries['winery'].unique()[0:3])]
    avg_price = selected_wineries['price'].mean()
    wines= ("The least expensive " + variety_prediction + "s are from these wineries: " + wineries['winery'].unique()[0] + ", " \
        + wineries['winery'].unique()[1] + ", or " + wineries['winery'].unique()[2] +". Avg Price: $%.2f" % avg_price)
    output_tweets.append(wines)
    
    return output_tweets

In [12]:
def tweetBack (out_tweets, target):
    count = 0 
    while count < len(out_tweets):
        for i in out_tweets:
            api.update_status(i + " " + target)
            count+= 1 

In [4]:
test_desc = "Currant, Plum, Black Cherry & Spice, with notes of Olive, Vanilla Mint, Tobacco, Toasty Cedar, Anise, Pepper & Herbs"

wine = runModel(test_desc)
out = get_wine_variety_response(wine)

out

['Red Blend']


["The highest rated Red Blends are from these wineries: Giovanni Chiappini, Colgin, or Tenuta dell'Ornellaia. Avg Price: $103.11",
 'The least expensive Red Blends are from these wineries: Campo alla Sughera, Waterbrook, or Viticultori Associati Canicatti. Avg Price: $30.48']